<a href="https://colab.research.google.com/github/javierdealba/inmobibot/blob/main/Avance2_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avance 2. Ingeniería de características

En el siguiente avance se hará la ingeniería de características como si se estuviera haciendo un modelo predictivo del precio final de las propiedades según sus características. El modelo propuesto es un RAG, por lo que la ingeniería de características no tiene impacto en el proyecto, sin embargo se cumplirá con este caso hipotético por fines de la rúbrica.

In [1]:
import pandas as pd

propiedades = pd.read_csv("listado_propiedades.csv")

y = propiedades["precio"]
columnas_descartables = ["direccion", "descripcion", "alias", "precio"]
propiedades_filtradas = propiedades.drop(columnas_descartables, axis=1)

# Se rellenan las características de manera distinta al avance 1,
# debido a lo asumido para esta entrega
propiedades_filtradas["niveles"] = propiedades_filtradas["niveles"].fillna(1)
propiedades_filtradas["recamaras"] = propiedades_filtradas["recamaras"].fillna(0)
propiedades_filtradas["banos"] = propiedades_filtradas["banos"].fillna(0)
propiedades_filtradas["medios_banos"] = propiedades_filtradas["medios_banos"].fillna(0)
propiedades_filtradas["cuarto_de_servicio"] = propiedades_filtradas["cuarto_de_servicio"].fillna(0)
propiedades_filtradas["estacionamientos"] = propiedades_filtradas["estacionamientos"].fillna(0)
propiedades_filtradas["construccion"] = propiedades_filtradas["construccion"].fillna(0)
propiedades_filtradas["frente"] = propiedades_filtradas["frente"].fillna(0)
propiedades_filtradas["fondo"] = propiedades_filtradas["fondo"].fillna(0)
propiedades_filtradas["amueblado"] = propiedades_filtradas["amueblado"].fillna(False)

propiedades_filtradas.head()

,tipo_de_propiedad,amueblado,niveles,recamaras,banos,medios_banos,cuarto_de_servicio,estacionamientos,terreno,frente,fondo,construccion,forma,uso_de_suelo,relieve,mantenimiento,estado,antiguedad,gravamen
0,casa,False,2.0,3.0,4.0,0.0,0.0,2.0,191.00,8.52,22.77,268.00,regular,habitacional,plano,0,excelente,0,no
1,casa,False,2.0,3.0,2.0,1.0,0.0,2.0,102.00,6.00,17.00,102.00,regular,habitacional,plano,900,bueno,12,si
2,casa,False,3.0,4.0,4.0,0.0,0.0,2.0,165.31,8.00,20.50,235.00,regular,habitacional,plano,0,excelente,0,no
3,casa,False,2.0,3.0,3.0,1.0,1.0,4.0,535.00,11.50,31.18,485.00,regular,habitacional,plano,0,bueno,35,no
4,casa,False,3.0,4.0,3.0,1.0,1.0,2.0,114.70,7.00,16.38,175.24,regular,habitacional,plano,1200,excelente,0,no


##Construccion

Aquí se convierten las variables descritas como "Sí" y "No" a tipo booleano, cambiando el tipo de datos y haciéndolas más fácil de utilizar, así como más óptimas.

In [2]:
propiedades_filtradas["amueblado"] = propiedades_filtradas["amueblado"].replace(["si", "no"],[True, False])
propiedades_filtradas["gravamen"] = propiedades_filtradas["gravamen"].replace(["si", "no"],[True, False])

# Validamos que ahora son datos booleanos
propiedades_filtradas[["amueblado", "gravamen"]].value_counts()

amueblado  gravamen
False      False       35
           True         8
True       False        1
Name: count, dtype: int64

In [3]:
propiedades_filtradas["uso_de_suelo"].unique()

array(['habitacional', 'comercial, habitacional', 'Habitacional',
       'habitacional, comercial y turistico', 'comercial ',
       'habitacional, turistico ', 'comercial, habitacional, turistico'],
      dtype=object)

En lugar de hacer one hot encoding a las característica de uso de suelo, manualmente se crearán variables booleanas que incluyan si es habitacional, comercial y/o turístico. Esto se hace debido a la falta de consistencia que existe en los datos y la sencillez con la que se puede realizar. Por ejemplo, no se opta por usar un multi label binarizer debido a los distintos valos de separación que existen y la discrepancia entre mayúsculas y minúsculas. Sí se pudiera hacer de esta manera, pero en este caso, es más sencillo realizarlo manualmente.

In [4]:
propiedades_filtradas["uso_habitacional"] =  propiedades_filtradas["uso_de_suelo"].apply(lambda x: "habitacional" in x.lower())
propiedades_filtradas["uso_comercial"] =  propiedades_filtradas["uso_de_suelo"].apply(lambda x: "comercial" in x.lower())
propiedades_filtradas["uso_turistico"] =  propiedades_filtradas["uso_de_suelo"].apply(lambda x: "turistico" in x.lower())

print(propiedades_filtradas[["uso_de_suelo", "uso_habitacional", "uso_comercial", "uso_turistico"]].tail())

                          uso_de_suelo  uso_habitacional  uso_comercial  \
39                        habitacional              True          False   
40  comercial, habitacional, turistico              True           True   
41  comercial, habitacional, turistico              True           True   
42            habitacional, turistico               True          False   
43                        habitacional              True          False   

    uso_turistico  
39          False  
40           True  
41           True  
42           True  
43          False  


##Normalización

Aquí se harán tranformaciones a través de One Hot Encoding y de Standard Scalling a las variables cualitativas y cuantitativas respectivamente, posteriormente se hará un análisis de compnentes principales, para ver si vale la pena eliminar alguna o algunas variables.

In [5]:
propiedades_filtradas = propiedades_filtradas.drop("uso_de_suelo", axis=1)
dtypes = propiedades_filtradas.dtypes
dtypes

tipo_de_propiedad      object
amueblado                bool
niveles               float64
recamaras             float64
banos                 float64
medios_banos          float64
cuarto_de_servicio    float64
estacionamientos      float64
terreno               float64
frente                float64
fondo                 float64
construccion          float64
forma                  object
relieve                object
mantenimiento           int64
estado                 object
antiguedad              int64
gravamen                 bool
uso_habitacional         bool
uso_comercial            bool
uso_turistico            bool
dtype: object

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

categoricas = dtypes[dtypes == "object"].index.tolist()
numericas = dtypes[dtypes.apply(lambda x: x.name not in ["object", "bool"])].index.tolist()
todas = dtypes.index.tolist()

transformador_categorico = OneHotEncoder(handle_unknown="ignore")
transformador_numerico = StandardScaler()

column_transformer = ColumnTransformer(
    transformers=[
        ("categorico", transformador_categorico, categoricas),
        ("numerico", transformador_numerico, numericas),
        ('pca', PCA(), numericas),
    ],
    remainder="passthrough"
)

pipeline = Pipeline([
    ("column_transformer", column_transformer)
])

propiedades_transformadas = pipeline.fit_transform(propiedades_filtradas, y)

## Selección / Extracción

In [7]:
nombres_columnas_transformadas = pipeline.named_steps['column_transformer'].get_feature_names_out()
componentes_pca = pipeline.named_steps['column_transformer'].named_transformers_['pca'].components_
componentes_df = pd.DataFrame(componentes_pca, columns=numericas, index=['Componente_'+str(i+1) for i in range(componentes_pca.shape[0])])

componentes_df

,niveles,recamaras,banos,medios_banos,cuarto_de_servicio,estacionamientos,terreno,frente,fondo,construccion,mantenimiento,antiguedad
Componente_1,-0.000032,-0.000106,-0.000072,-0.000031,-0.000013,-0.000079,0.999598,0.004409,0.005493,-0.007246,-0.026504,-0.000403
Componente_2,-0.000030,0.000046,0.000145,0.000014,0.000083,0.000008,0.026536,-0.000355,-0.000785,0.003608,0.999641,-0.000925
Componente_3,0.001796,0.009802,0.002766,0.007612,0.000474,0.012031,0.007092,-0.002036,0.013695,0.999309,-0.003760,0.028556
Componente_4,0.001338,-0.001720,0.002494,-0.003651,0.000127,-0.002254,-0.005528,-0.008853,0.999832,-0.013704,0.000981,0.003135
Componente_5,0.002617,-0.005584,0.002686,-0.008676,0.000682,-0.006355,-0.004424,0.999575,0.008703,0.001441,0.000496,0.024290
Componente_6,-0.008395,-0.002962,-0.018809,-0.025687,-0.000531,-0.014988,0.000351,-0.024454,-0.003806,-0.028061,0.001007,0.998640
Componente_7,-0.035616,0.517019,-0.143679,0.564255,-0.028784,0.624999,-0.000080,0.011703,0.004568,-0.017092,0.000078,0.022230
Componente_8,-0.273503,-0.513113,-0.748444,0.288839,-0.131445,-0.029515,-0.000095,0.002500,0.002494,0.006094,0.000105,-0.010759
Componente_9,-0.487340,-0.424401,0.314597,-0.222165,0.254355,0.607732,-0.000002,-0.000271,-0.000374,-0.001688,-0.000055,0.004056
Componente_10,-0.644145,0.402901,-0.076667,-0.347128,-0.535204,-0.098276,-0.000009,0.001259,0.000414,0.001961,0.000003,-0.016265


Tras ver el análisis de componentes principales, se puede determinar que la variable que tiene mayor impacto directo con el precio de manera general, es el tamaño del terreno, seguido del costo de mantenimiento y luego de los metros de construcción.
Es lógico que las propiedades con mayor terreno y mayor construcción tengan un mayor costo. También, tiene sentido que si el costo de mantenimiento es elevado, puede indicar un costo más elevando, sin embargo, este número ayudaría solo a estimar el costo de una propiedad según el costo que ya se estableció de mantenimiento, y no que si se aumenta el costo de mantenimiento, forzosamente aumentaría el precio de la propiedad.

A partir de ahi, cada una puede tener un impacto distinto, según las combinaciones de variables, y se deciden eliminar las variables de:


*   Niveles
*   Recámaras
*  Baños
* Medios baños
* Cuarto de servicio
* Estacionamiento


In [8]:
columnas_descartables = ["niveles", "recamaras", "banos", "medios_banos", "cuarto_de_servicio", "estacionamientos"]
propiedades_filtradas = propiedades_filtradas.drop(columnas_descartables, axis=1)
propiedades_filtradas.head()

,tipo_de_propiedad,amueblado,terreno,frente,fondo,construccion,forma,relieve,mantenimiento,estado,antiguedad,gravamen,uso_habitacional,uso_comercial,uso_turistico
0,casa,False,191.00,8.52,22.77,268.00,regular,plano,0,excelente,0,False,True,False,False
1,casa,False,102.00,6.00,17.00,102.00,regular,plano,900,bueno,12,True,True,False,False
2,casa,False,165.31,8.00,20.50,235.00,regular,plano,0,excelente,0,False,True,False,False
3,casa,False,535.00,11.50,31.18,485.00,regular,plano,0,bueno,35,False,True,False,False
4,casa,False,114.70,7.00,16.38,175.24,regular,plano,1200,excelente,0,False,True,False,False


## Conclusiones

En términos de CRISP ML, después de esta práctica los datos ya se encuentran limpios y son coherentes, están integrados y enriquecidos, y están documentados de manera correcta. Se tomó la decisión de filtrar ciertas variables debido a su bajo impacto en la variable predictiva, que para fines de esta práctica, es el precio. En un futuro, todas las variables se van a considerar, ya que el fin del modelo RAG será que el chatbot pueda brindar la información adecuada a los clientes de lo que pueda preguntar. De la misma manera, no será necesaria la normalización de las variables, sin embargo, estas se normalizaron según standard scaling y one hot encoding. Finalmente, las variables creadas tampoco tendrán un valor para el RAG, aunque para un teórico modelo predictivo, estos nuevos formatos en las variables pueden ser de gran ayuda.

En conclusión, el paso de preparación de los datos para un modelo RAG consistía más en la creación de la base de datos que ya se hizo, sin embargo en este ejercicio simulando una predicción, sí se requirió construcción de variables, normalización y selección y extracción de variables.